In [2]:
import pandas as pd

# 1. Loading Raw data


df = pd.read_csv("YWMB47D  Bank of England  Database.csv")   # file is in the same folder
df = df.iloc[:, :2]                 # keep first 2 columns only
df.columns = ["Date", "Rate"]       # rename for convenience


# 2. Parsing dates and sorting 


df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)
df = df.sort_values("Date")
df = df.set_index("Date")


# 3. Making daily series

daily = df["Rate"].asfreq("D", method="ffill")

# 4. Resampling to monthy (END-OF-MONTH RATE)


monthly = daily.resample("M").last()

# Filter the period you need (adjust if needed)
monthly = monthly.loc["2015-01-01":"2025-10-31"]

monthly_df = monthly.to_frame(name="BankRate")

#Saving the file 

monthly_df.to_csv("bank_rate_monthly.csv", index_label="Date")

print("Done! Clean monthly Bank Rate saved as bank_rate_monthly.csv")
print(monthly_df.head())
print(monthly_df.tail())


Done! Clean monthly Bank Rate saved as bank_rate_monthly.csv
            BankRate
Date                
2015-01-31       0.5
2015-02-28       0.5
2015-03-31       0.5
2015-04-30       0.5
2015-05-31       0.5
            BankRate
Date                
2025-05-31      4.25
2025-06-30      4.25
2025-07-31      4.25
2025-08-31      4.00
2025-09-30      4.00


/var/folders/yw/t9jz6sd55w71t2th98wmv_180000gn/T/ipykernel_24383/1876296201.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)
/var/folders/yw/t9jz6sd55w71t2th98wmv_180000gn/T/ipykernel_24383/1876296201.py:29: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = daily.resample("M").last()


In [3]:
import pandas as pd
import re

# 1. Loading the full CPI file (with metadata, annual, quarterly, monthly)
df = pd.read_csv("series-121225.csv", header=None, names=["Key", "Value"])

# 2. Keeping only rows that look like: "YYYY MON"
#    e.g. "2015 JAN", "2022 OCT", "2025 MAR"
pattern = r"^(19|20)\d{2}\s+(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)$"
mask = df["Key"].str.match(pattern)

cpi = df[mask].copy()

# 3. Converting "2015 JAN" → proper datetime (first day of month)
cpi["Date"] = pd.to_datetime(cpi["Key"], format="%Y %b")

# 4. Converting the Value column to float and rename to Inflation
cpi["Inflation"] = cpi["Value"].astype(float)

# 5. Keeping only Date + Inflation, sort by Date
cpi = cpi[["Date", "Inflation"]].sort_values("Date")
cpi = cpi.set_index("Date")

# 6. Filtering to your analysis window: Jan 2015 – Oct 2025
cpi = cpi.loc["2015-01-01":"2025-10-01"]

# 7. Saving clean monthly CPI file
cpi.to_csv("cpi_monthly.csv", index_label="Date")

print(cpi.head())
print(cpi.tail())
print("Saved as cpi_monthly.csv")


            Inflation
Date                 
2015-01-01        0.5
2015-02-01        0.4
2015-03-01        0.3
2015-04-01        0.3
2015-05-01        0.4
            Inflation
Date                 
2025-06-01        4.1
2025-07-01        4.2
2025-08-01        4.1
2025-09-01        4.1
2025-10-01        3.8
Saved as cpi_monthly.csv


In [10]:
import pandas as pd

# 1. LOADING RAW CSV (change the filename to your actual one)
df_raw = pd.read_csv("GDP full.csv", header=None, names=["Period", "GDP_QoQ"])

# 2. KEEPING ONLY QUARTERLY ROWS LIKE "2015 Q1"
mask_quarter = df_raw["Period"].astype(str).str.match(r"^\d{4} Q[1-4]$")
df = df_raw[mask_quarter].copy()

# 3. FIXING FORMAT: "2015 Q1" → "2015Q1" SO PANDAS CAN PARSE IT
df["QuarterStr"] = df["Period"].str.replace(" ", "", regex=False)   # "2015Q1"

# 4. CONVERTING TO QUARTERLY PERIOD, THEN TO END-OF-QUARTER DATE
df["Quarter"] = pd.PeriodIndex(df["QuarterStr"], freq="Q-DEC")
df["Date"] = df["Quarter"].dt.to_timestamp(how="end")  # e.g. 2015Q1 → 2015-03-31

# 5. FILTERNG FROM 2015 Q1 TO 2025 Q3
start_q = pd.Period("2015Q1", freq="Q-DEC")
end_q   = pd.Period("2025Q3", freq="Q-DEC")

df_filtered = df[(df["Quarter"] >= start_q) & (df["Quarter"] <= end_q)].copy()

# 6. KEEPING NICE COLUMNS & SORT
df_filtered = df_filtered[["Date", "Quarter", "GDP_QoQ"]].sort_values("Date")

# 7. SAVING CLEAN CSV FOR THE THESIS
df_filtered.to_csv("gdp_qoq_2015_2025Q3_clean.csv", index=False)

# 8. QUICK CHECK
print(df_filtered.head())
print(df_filtered.tail())


                             Date Quarter GDP_QoQ
247 2015-03-31 23:59:59.999999999  2015Q1     0.3
248 2015-06-30 23:59:59.999999999  2015Q2     0.5
249 2015-09-30 23:59:59.999999999  2015Q3     0.4
250 2015-12-31 23:59:59.999999999  2015Q4     0.6
251 2016-03-31 23:59:59.999999999  2016Q1     0.5
                             Date Quarter GDP_QoQ
285 2024-09-30 23:59:59.999999999  2024Q3     0.2
286 2024-12-31 23:59:59.999999999  2024Q4     0.2
287 2025-03-31 23:59:59.999999999  2025Q1     0.7
288 2025-06-30 23:59:59.999999999  2025Q2     0.3
289 2025-09-30 23:59:59.999999999  2025Q3     0.1


In [13]:
import pandas as pd

# Checking each file's date format
files = {
    'FTSE': 'FTSE 100 Historical Data.csv',
    'GBP/USD': 'GBP_USD Historical Data .csv', 
    'CPI': 'cpi_monthly.csv',
    'Bank Rate': 'bank_rate_monthly.csv',
    'GDP': 'GDP_Q.csv'
}

for name, file in files.items():
    try:
        df = pd.read_csv(file, nrows=5)
        print(f"\n{name}:")
        print(f"  Columns: {list(df.columns)}")
        print(f"  Date sample: {df['Date'].head(2).tolist() if 'Date' in df.columns else 'No Date column'}")
        print(f"  Total rows: {len(pd.read_csv(file))}")
    except:
        print(f"\n{name}: File not found or error")


FTSE:
  Columns: ['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']
  Date sample: ['12/01/2025', '11/01/2025']
  Total rows: 131

GBP/USD:
  Columns: ['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']
  Date sample: ['12/01/2025', '11/01/2025']
  Total rows: 132

CPI:
  Columns: ['Date', 'Inflation']
  Date sample: ['2015-01-01', '2015-02-01']
  Total rows: 130

Bank Rate:
  Columns: ['Date', 'BankRate']
  Date sample: ['2015-01-31', '2015-02-28']
  Total rows: 129

GDP:
  Columns: ['Date', 'Quarter', 'GDP_QoQ']
  Date sample: ['2015-03-31 23:59:59.999999999', '2015-06-30 23:59:59.999999999']
  Total rows: 43


In [25]:
import pandas as pd

# Reading and processing each file
ftse = pd.read_csv('FTSE 100 Historical Data.csv')
ftse['Date'] = pd.to_datetime(ftse['Date']) + pd.offsets.MonthEnd(0)
ftse['FTSE_Return'] = ftse['Change %'].str.replace('%', '').astype(float)

gbp = pd.read_csv('GBP_USD Historical Data .csv')
gbp['Date'] = pd.to_datetime(gbp['Date']) + pd.offsets.MonthEnd(0)
gbp['GBPUSD_Return'] = gbp['Change %'].str.replace('%', '').astype(float)

cpi = pd.read_csv('cpi_monthly.csv')
cpi['Date'] = pd.to_datetime(cpi['Date']) + pd.offsets.MonthEnd(0)

bank = pd.read_csv('bank_rate_monthly.csv')
bank['Date'] = pd.to_datetime(bank['Date']) + pd.offsets.MonthEnd(0)

gdp = pd.read_csv('GDP_Q.csv')
gdp['Date'] = pd.to_datetime(gdp['Date'])
gdp['Quarter'] = gdp['Date'].dt.to_period('Q')

# Merge
df = ftse[['Date', 'FTSE_Return']]
df = df.merge(gbp[['Date', 'GBPUSD_Return']], on='Date')
df = df.merge(cpi[['Date', 'Inflation']], on='Date')
df = df.merge(bank[['Date', 'BankRate']], on='Date')

# Adding GDP
df['Quarter'] = df['Date'].dt.to_period('Q')
df = df.merge(gdp[['Quarter', 'GDP_QoQ']], on='Quarter', how='left')

# Finalize
df = df[['Date', 'Quarter', 'FTSE_Return', 'GBPUSD_Return', 'Inflation', 'BankRate', 'GDP_QoQ']]
df = df.sort_values('Date')
df.to_csv('thesis_data_final.csv', index=False)

print(f"✅ Saved! {len(df)} rows")
print(df.head())

✅ Saved! 129 rows
        Date Quarter  FTSE_Return  GBPUSD_Return  Inflation  BankRate  GDP_QoQ
0 2015-01-31  2015Q1         2.79          -3.26        0.5       0.5      0.3
1 2015-02-28  2015Q1         2.92           2.43        0.4       0.5      0.3
2 2015-03-31  2015Q1        -2.50          -3.99        0.3       0.5      0.3
3 2015-04-30  2015Q2         2.77           3.60        0.3       0.5      0.5
4 2015-05-31  2015Q2         0.34          -0.40        0.4       0.5      0.5


/var/folders/yw/t9jz6sd55w71t2th98wmv_180000gn/T/ipykernel_24383/1947434253.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ftse['Date'] = pd.to_datetime(ftse['Date']) + pd.offsets.MonthEnd(0)
/var/folders/yw/t9jz6sd55w71t2th98wmv_180000gn/T/ipykernel_24383/1947434253.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  gbp['Date'] = pd.to_datetime(gbp['Date']) + pd.offsets.MonthEnd(0)


In [17]:
import pandas as pd

df = pd.read_csv('thesis_data_final.csv')
print(f"Total months: {len(df)}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
print(df.isnull().sum())  # Should all be 0
print(df.describe())  # Checking summary stats

Total months: 128
Date range: 2015-02-28 to 2025-09-30
Date             0
Quarter          0
FTSE_Return      0
GBPUSD_Return    0
Inflation        0
BankRate         0
GDP_QoQ          0
dtype: int64
       FTSE_Return  GBPUSD_Return   Inflation    BankRate     GDP_QoQ
count   128.000000     128.000000  128.000000  128.000000  128.000000
mean      0.315078      -0.059375    3.039062    1.635547    0.435937
std       3.453478       2.421718    2.458321    1.926291    4.209976
min     -13.810000      -8.090000    0.200000    0.100000  -19.900000
25%      -1.812500      -1.737500    1.300000    0.250000    0.100000
50%       0.815000      -0.085000    2.300000    0.500000    0.400000
75%       2.542500       1.842500    4.025000    3.625000    0.700000
max      12.350000       5.560000    9.600000    5.250000   17.000000


In [21]:
import pandas as pd

# Loading and parsing dates
ftse = pd.read_csv('FTSE 100 Historical Data.csv')
ftse['Date'] = pd.to_datetime(ftse['Date'], format='%m/%d/%Y')
ftse['Price'] = ftse['Price'].str.replace(',', '').astype(float)

# 1. SORT ASCENDING (oldest to newest)
ftse = ftse.sort_values('Date', ascending=True)

# 2. NOW calculating returns
ftse['FTSE_Return_Correct'] = ftse['Price'].pct_change() * 100

print("Correctly calculated returns (first 5):")
print(ftse[['Date', 'Price', 'FTSE_Return_Correct']].head())

print("\nCheck Feb 2015:")
feb2015 = ftse[ftse['Date'].dt.strftime('%Y-%m') == '2015-02']
print(feb2015[['Date', 'Price', 'FTSE_Return_Correct']])

Correctly calculated returns (first 5):
          Date    Price  FTSE_Return_Correct
130 2015-02-01  6946.66                  NaN
129 2015-03-01  6773.04            -2.499331
128 2015-04-01  6960.63             2.769657
127 2015-05-01  6984.43             0.341923
126 2015-06-01  6520.98            -6.635473

Check Feb 2015:
          Date    Price  FTSE_Return_Correct
130 2015-02-01  6946.66                  NaN


In [23]:
import pandas as pd

# Loading original FTSE
ftse = pd.read_csv('FTSE 100 Historical Data.csv')
print("Original data first 5 rows:")
print(ftse[['Date', 'Price']].head())

# Parsing dates
ftse['Date_parsed'] = pd.to_datetime(ftse['Date'], format='%m/%d/%Y')
ftse['Price_clean'] = ftse['Price'].str.replace(',', '').astype(float)

# Sorting ascending
ftse = ftse.sort_values('Date_parsed', ascending=True)

print("\nSorted data (first 10 rows):")
print(ftse[['Date', 'Date_parsed', 'Price_clean']].head(10))

# Manually checking Jan and Feb 2015
jan = ftse[ftse['Date_parsed'].dt.strftime('%Y-%m') == '2015-01']
feb = ftse[ftse['Date_parsed'].dt.strftime('%Y-%m') == '2015-02']

print("\nJan 2015 data:")
print(jan[['Date', 'Date_parsed', 'Price_clean']])
print("\nFeb 2015 data:")
print(feb[['Date', 'Date_parsed', 'Price_clean']])

# Manual calculation
if len(jan) > 0 and len(feb) > 0:
    jan_price = jan['Price_clean'].iloc[0]
    feb_price = feb['Price_clean'].iloc[0]
    manual_return = (feb_price / jan_price - 1) * 100
    print(f"\nManual calculation:")
    print(f"Jan price: {jan_price}")
    print(f"Feb price: {feb_price}")
    print(f"Return = ({feb_price} / {jan_price} - 1) * 100 = {manual_return:.4f}%")

Original data first 5 rows:
         Date     Price
0  12/01/2025  9,664.00
1  11/01/2025  9,720.51
2  10/01/2025  9,717.25
3  09/01/2025  9,350.43
4  08/01/2025  9,187.34

Sorted data (first 10 rows):
           Date Date_parsed  Price_clean
130  02/01/2015  2015-02-01      6946.66
129  03/01/2015  2015-03-01      6773.04
128  04/01/2015  2015-04-01      6960.63
127  05/01/2015  2015-05-01      6984.43
126  06/01/2015  2015-06-01      6520.98
125  07/01/2015  2015-07-01      6696.28
124  08/01/2015  2015-08-01      6247.94
123  09/01/2015  2015-09-01      6061.61
122  10/01/2015  2015-10-01      6361.09
121  11/01/2015  2015-11-01      6356.09

Jan 2015 data:
Empty DataFrame
Columns: [Date, Date_parsed, Price_clean]
Index: []

Feb 2015 data:
           Date Date_parsed  Price_clean
130  02/01/2015  2015-02-01      6946.66
